  <img src = "https://drive.google.com/uc?export=view&id=1K9OCVYN1o-SI5c11HV1z2R-ruaN4IQet" alt = "Encabezado MLDS unidad 1" width = "100%">  </img>

# **Taller 1 : SQLite y Pandas**
---

En este taller se evaluarán las habilidades adquiridas usando _Python_ en el manejo de la librería _SQLite_. Usted deberá realizar algunas operaciones de consulta de bases de datos con la librería `sqlite3` de la librería estándar de _Python_ para una base de datos real con información histórica del campeonato de _Fórmula 1_.

> **Nota:** Esta tarea va a ser calificada en la plataforma **[UNCode](https://juezun.github.io/)**. Para esto, en cada ejercicio se indicará si es calificable o no, también los lugares donde debe escribir su código sin modificar lo demás con un aproximado de cantidad de líneas a escribir. No se preocupe si su código toma más líneas, esto es simplemente un aproximado destinado a que pueda replantear su estrategia si el código está tomando más de las esperadas. No es un requisito estricto y soluciones más largas también son válidas. Al finalizar, para realizar el envío (*submission*), descargue el notebook como un archivo **`.ipynb`** y haga su entrega a través de **Edunext**.

Ejecute la siguiente celda para importar las librerías.

In [ ]:
# Importar librerías
import sqlite3
import numpy as np
import pandas as pd

In [ ]:
#TEST_CELL
# !python --version

import platform
print('SQLite (Python):', platform.python_version())
print('NumPy', np.__version__)
print('Pandas', pd.__version__)

Este material fue realizado con las siguientes versiones:

- Python: 3.7.10
- SQLite (Python): 3.7.10
- NumPy: 1.19.5
- Pandas: 1.1.5

## **Base de datos**
---

En esta tarea se va a utilizar el conjunto de datos [**Formula 1 Race Data**](https://www.kaggle.com/cjgdev/formula-1-race-data-19502017) de _Kaggle_, y en particular, una [versión](https://www.kaggle.com/davidcochran/formula-1-race-data-sqlite) del conjunto de datos construido para su uso en el formato del estándar _SQLite_. Este fue recopilado de la plataforma [**Ergast**](http://ergast.com/mrd/) y contiene información de la competición entre los años $1950$ y $2017$ y está compuesto por varias tablas para los conductores, circuitos, fabricantes y carreras, entre otros.

Para cargar el conjunto de datos realizaremos una conexión a un archivo local con extensión `.sqlite`. Ejecute la siguiente celda para descargar el archivo:

> **Nota**: El archivo `formula1.sqlite` estará en el ambiente de _UNCode_, no se preocupe por ello.

In [ ]:
# TEST_CELL
!wget 'https://drive.google.com/uc?export=view&id=1DupYqI2NnceaeVkGyvkMOVmcJsY1rKg9' -O formula1.sqlite

### Comandos PRAGMA de SQLite

---

La sentencia **PRAGMA** es una extensión específica de *SQLite* utilizada para modificar el comportamiento o las operaciones de la librería de SQLite mediante el llamado a funciones especiales y/o para consultar dicha librería en busca de datos internos (que no sean tablas).

Existen 4 categorías de pragmas disponibles:

*   Los pragmas utilizados para consultar el esquema de la base de datos actual.
*   Los usados para analizar y depurar la biblioteca. Así se logra verificar que los archivos de la base de datos no estén corruptos.
*   Los pragmas empleados para consultar o modificar los valores de las versiones de las bases de datos: la versión del esquema, la cual manipulada por SQLite internamente, y la versión del usuario, que es utilizada por las aplicaciones para cualquier propósito.
*   Por último, los pragmas empleados para, como mencionamos anteriormente, para consultar el modo de funcionamiento actual de la librería de SQLite o modificarlo.

> Si se utiliza una sentencia **PRAGMA** que no sea reconocida por el lenguaje, *SQLite* simplemente la ignora y no efectúa la acción del pragma específicado. De igual manera, si hay un error tipográfico al momento de declarar un pragma, la librería no informa al usuario del error, solo continúa con la ejecución del resto de sentencias.

Algunas de las funciones PRAGMA más utilizadas son:

*   `foreign_key_check`: permite verificar cuales restricciones de las llaves foráneas existentes en la base de datos están siendo infringidas o vulneradas. En caso de que se requieran verifican solo las llaves foráneas de una tabla en específico, se adiciona el nombre de la tabla como argumento a la función ➡️ `foreign_key_check (table-name)`.

*   `foreign_key_list(table-name)`: este retorna una lista de todas las restricciones de llaves foráneas que contiene la tabla específicada.

*   `foreign_keys = boolean`: permite consultar, establecer o eliminar la aplicación de las restricciones de las llaves foráneas en la base de datos. Tenga en cuenta que, una vez *modificadas* dichas llaves, se ven afectados todas las consultas o declaraciones preparadas con antelación.

*   `query_only = boolean`: esta sentencia previene que se modifiquen los datos de la base de datos. Cuando este pragma está activo, cualquier intento con las sentencias `CREATE`, `DROP`, `DELETE`, `UPDATE` o `INSERT` resultarán en un mensaje de error `SQLITE_READONLY` indicando que no se tienen permisos de escritura.

*   `table_info(table-name)`: retorna una fila por cada columna en la tabla indicada. Cada fila incluye el nombre de la columna, el tipo de datos que acepta, si la misma puede contener un valor nulo o no, el valor predeterminado de la columna. Además, incluye un campo *pk* el cual contiene un 0 si la columna no es la llave primaria, en caso contario, contendrá el valor del índice que ocupe la columna de la llave primaria en la tabla.

> Los PRAGMAs que retornan un resultado y no involucran una modificación en los datos pueden ser utilizados junto a la sentencia (`SELECT`).

Ahora, iniciamos la conexión:

In [ ]:
# TEST_CELL
# Iniciamos y configuramos la conexión con la
# base de datos en modo de solo consulta.
conn = sqlite3.connect("formula1.sqlite")
conn.execute('PRAGMA query_only = ON')
conn.commit()

En este caso, trabajaremos con 4 de las tablas disponibles. Estas son la tabla de conductores (`drivers`), circuitos (`circuits`), carreras (`races`) y resultados (`results`). Tenemos el siguiente modelo de datos:

<img src="https://drive.google.com/uc?export=view&id=1g5LKZFSNuwbazl168jCNVzg9yVyIndNO" width="70%" alt="Diagrama base de datos"></img>

Veamos los campos de cada una de ellas:

### **Tabla de conductores.**

In [ ]:
#TEST_CELL
# Campos de la tabla de conductores.
query = "PRAGMA table_info(drivers);"
pd.read_sql(query, conn)

### **Tabla de circuitos.**

In [ ]:
#TEST_CELL
# Campos de la tabla de circuitos.
query = "PRAGMA table_info(circuits);"
pd.read_sql(query, conn)

### **Tabla de carreras.**

In [ ]:
#TEST_CELL
# Campos de la tabla de carreras.
query = "PRAGMA table_info(races);"
pd.read_sql(query, conn)

### **Tabla de resultados**

In [ ]:
#TEST_CELL
# Campos de la tabla de resultados.
query = "PRAGMA table_info(results);"
pd.read_sql(query, conn)

## **1. Circuitos del Hemisferio Sur**
---

La primera tabla que consideraremos es la que contiene la lista de las pistas o circuitos (`circuits`) de la competición. En esta, se almacena información geográfica del circuito, como el país, ciudad y coordenadas geográficas de su ubicación exacta.

> Las [coordenadas geográficas](https://es.wikipedia.org/wiki/Coordenadas_geogr%C3%A1ficas) permiten codificar la posición en el mundo de un punto determinado y están compuestas por los valores de **latitud** y **longitud**. La latitud representa el ángulo de la ubicación con respecto al plano del ecuador (con $-90°$ para el polo sur y $90°$ para el polo norte) y la longitud representa el ángulo con respecto al meridiano de _Greenwich_ (con valores de  $-180°$ hacia el occidente y $180°$ hacia el oriente).

En este ejercicio usted deberá completar la función `south_circuits`, que retorne una cadena de texto con una **consulta SQL válida** que retorne el **país**, **ciudad** y **nombre** de los circuitos ubicados en el **hemisferio sur**.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA south_circuits:

def south_circuits():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.

    query = """
    SELECT
        ...
    FROM
        ...
    WHERE
        ...
    ;
    """
    return query

Use la siguiente celda para probar su consulta:

In [ ]:
#TEST_CELL
from IPython.display import display
try:
    display(pd.read_sql(south_circuits(), conn))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>country</th>
      <th>location</th>
      <th>name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Australia</td>
      <td>Melbourne</td>
      <td>Albert Park Grand Prix Circuit</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Brazil</td>
      <td>SÃ£o Paulo</td>
      <td>AutÃ_dromo JosÃ© Carlos Pace</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Argentina</td>
      <td>Buenos Aires</td>
      <td>AutÃ_dromo Juan y Oscar GÃ¡lvez</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Australia</td>
      <td>Adelaide</td>
      <td>Adelaide Street Circuit</td>
    </tr>
    <tr>
      <th>4</th>
      <td>South Africa</td>
      <td>Midrand</td>
      <td>Kyalami</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Brazil</td>
      <td>Rio de Janeiro</td>
      <td>AutÃ_dromo Internacional Nelson Piquet</td>
    </tr>
    <tr>
      <th>6</th>
      <td>South Africa</td>
      <td>Eastern Cape Province</td>
      <td>Prince George Circuit</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique los campos seleccionados con su consulta y asegúrese de indicarlas en minúsculas. Estas deben ser, en ese orden:
  * `country`: país.
  * `location`: ciudad.
  * `name`: nombre del circuito.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Los puntos ubicados en el hemisferio sur son aquellos con valores negativos de **latitud**. Utilice una expresión `WHERE` para condicionar la consulta a los valores negativos del campo `lat` de la tabla.

## **2. Pilotos de una Nacionalidad Específica**
---

Otra de las tablas disponibles en la base de datos es la tabla de los pilotos (`drivers`) con información básica como sus códigos de identificación, nombres, apellidos, fecha de nacimiento, nacionalidad y una url de *Wikipedia* con más información.

> Para conocer las nacionalidades únicas de la base de datos, podemos utilizar la palabra clave `DISTINCT`, que permite obtener los valores sin repeticiones de un campo determinado.

In [ ]:
#TEST_CELL
# Nacionalidades únicas del campo 'nationality'.
query = """
SELECT DISTINCT
    nationality
FROM
    drivers
;
"""
pd.read_sql(query, conn)

En este ejercicio usted deberá completar la función `drivers_by_nation`, que reciba una cadena de texto con una nacionalidad (correspondiente a valores del campo `nationality`) y retorne una cadena de texto con una consulta SQL válida que permita obtener el **nombre**, **apellido** y **url** de los pilotos de dicha nacionalidad.

**Parámetros de entrada:**

* `nationality`: cadena de texto con la nacionalidad de los conductores que se desean obtener, que corresponde a alguno de los valores de la tabla anterior.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA drivers_by_nation:

def drivers_by_nation(nationality):
    """
    Entradas:
        nationality: cadena de texto con la nacionalidad de los conductores.
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query.format(nationality)

Use la siguiente celda para probar su código:

In [ ]:
#TEST_CELL
from IPython.display import display
try:
    display(pd.read_sql(drivers_by_nation('Colombian'), conn))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**

| | forename | surname | url |
|--- | --- | --- | --- |
| 0 | Juan | Pablo Montoya | http://en.wikipedia.org/wiki/Juan_Pablo_Montoya |
| 1 | Roberto | Guerrero | http://en.wikipedia.org/wiki/Roberto_Guerrero |
| 2 | Ricardo | Londo̱o | http://en.wikipedia.org/wiki/Ricardo_Londo%C3%B1o |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique los campos seleccionados con su consulta y asegúrese de indicarlas en minúsculas. Estas deben ser, en ese orden:
  * `forename`: nombres.
  * `surname`: apellidos.
  * `url`: url de _Wikipedia_.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Utilice una expresión `WHERE` para condicionar la consulta a los valores exactos del campo `nationality`. Utilice una cadena de texto [**f-string**](https://realpython.com/python-f-strings/) o métodos similares como la concatenación de cadenas de texto para usar el valor dado como argumento.

Es importante que tenga en cuenta el uso de comillas. De lo contrario se interpretará el valor como el nombre de una variable, en vez de una cadena de texto. La cadena de su respuesta debería quedar:

```sql
...
WHERE
  _______ = 'Colombian'
...
```

Si por el contrario queda como se muestra a continuación no se obtendrá un resultado correcto pues el motor intentará encontrar la variable `Colombian`.

```sql
...
WHERE
  _______ = Colombian
...
```

## **3. Número de Conductores por Nacionalidad**
---
En este caso también se utilizará la tabla de pilotos ( `drivers` ) utilizada en el ejercicio 2. Tabla que almacena atributos de cada piloto como su nombre, apellido, edad de nacimiento e inclusive una URL que apunta a *Wikipedia* donde se expande aún más la información.

> * Para conocer el número de conductores por nacionalidad se debe usar la función de agregación de SQL `COUNT()` que cuenta la cantidad de filas retornadas por una operación de selección `SELECT`
* Las funciones de agregación como `COUNT(), AVG(), SUM()`, entre otras, generan un campo (o columna) adicional  en las tablas retornadas por un *query*. Para este caso la función `COUNT()` genera un nuevo columna que debe ser renombrado en letras minúsculas y exactamente como `counts`


En este ejercicio usted deberá completar la función `number_drivers_nation()`, que retorne una cadena de texto con una **consulta SQL válida** que retorne **TODAS** las nacionalidades con una cuenta de la cantidad de **TODOS** los pilotos que hay en cada una de ellas.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA number_drivers_nation:

def number_drivers_nation():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query

Use la siguiente celda para probar su código:

In [ ]:
from IPython.display import display
try:
    display(pd.read_sql(number_drivers_nation(), conn).head(10))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**

|    | nationality       |   counts |
|---:|:------------------|---------:|
|  0 | American          |      157 |
|  1 | American-Italian  |        1 |
|  2 | Argentine         |       24 |
|  3 | Argentine-Italian |        1 |
|  4 | Australian        |       17 |
|  5 | Austrian          |       15 |
|  6 | Belgian           |       23 |
|  7 | Brazilian         |       31 |
|  8 | British           |      162 |
|  9 | Canadian          |       13 |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique el campo seleccionado con su consulta y asegúrese de indicarlo en minúscula. Este debe ser:
  * `nationality`: país
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Recuerde que para renombrar un campo en una tabla retornada por un *query* se debe usar, después del nombre del campo o de la función de agregación, la sentencia `AS` y posteriormente agregar el nombre deseado.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Su consulta debería incluir la sentencia `GROUP BY` al campo `nationality`, ya que debe hacer la cuenta de todos los pilotos por cada nacionalidad *diferente*.
</details>

## **4. Número de Ciudades donde hay Carreras por País**

Para este ejercicio consideraremos nuevamente la tabla (`circuits`) ya que esta es la que contiene la información acerca de la ubicación de cada carrera. En este caso necesitaremos el país y la ciudad donde ocurre cada una de estas.

> La función `COUNT` retorna el número de filas que coinciden con un criterio específico.

En este ejercicio usted debe completar la función `number_cities_country`, la cual deberá retornar una cadena de texto con una **consulta SQL válida** que contenga el **país** y la **cantidad de ciudades** de este donde se lleva a cabo al menos una carrera.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
def number_cities_country():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query

In [ ]:
from IPython.display import display
try:
    display(pd.read_sql(number_cities_country(), conn).head(10))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>country</th>
      <th>counts</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Argentina</td>
      <td>1</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Australia</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Austria</td>
      <td>2</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Azerbaijan</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Bahrain</td>
      <td>1</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Belgium</td>
      <td>3</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Brazil</td>
      <td>2</td>
    </tr>
    <tr>
      <th>7</th>
      <td>Canada</td>
      <td>3</td>
    </tr>
    <tr>
      <th>8</th>
      <td>China</td>
      <td>1</td>
    </tr>
    <tr>
      <th>9</th>
      <td>France</td>
      <td>7</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Las consultas deben indicarse en minúsculas. Estas deben seguir el siguiente orden:
  * `country`: país.
  * `counts`: cantidad de ciudades con carreras.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Verifique el resultado del query. Tenga en cuenta que SQL tiene la palabra reservada `AS` para crear un *alias* y así renombrar una columna o tabla al realizar una consulta.
</details>

## **5. Número de Carreras por Año**
---
El ejercicio cinco usa la tabla de carreras ( `races` ). Tabla que almacena campos de cada carrera como el año cuando ocurrió (`year`), el ID de cada carrera (`raceId`), el nombre (`name`), fecha (`date`) e inclusive también una URL (`url`) que apunta a *Wikipedia* donde se expande aún más la información. No olvide que en la sección *Bases de datos* puede ver el modelo usado para todas las tablas.

> * Para conocer el número de carreras por año se debe usar la función de agregación de SQL `COUNT()` que cuenta la cantidad de filas retornadas por una operación de selección `SELECT`.
* La función `COUNT()` suele ser usada en las consultas donde aparece una sentencia `GROUP BY`.
* Las funciones de agregación como `COUNT(), AVG(), SUM()`, entre otras, generan un campo (o columna) adicional  en las tablas retornadas por un *query*. Para este caso la función `COUNT()` genera una nuevo campo que debe ser renombrado en letras minúsculas y exactamente como `counts`.


En este ejercicio usted deberá completar la función `number_races_year()`, que retorne una cadena de texto con una **consulta SQL válida** que retorne **TODOS** las años con una cuenta de la cantidad de **TODAS** los carreras que sucedieron anualmente.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA number_races_year:

def number_races_year():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query

Use la siguiente celda para probar su código:

In [ ]:
from IPython.display import display
try:
    display(pd.read_sql(number_races_year(), conn))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**

|    |   year |   counts |
|---:|-------:|---------:|
|  0 |   1950 |        7 |
|  1 |   1951 |        8 |
|  2 |   1952 |        8 |
|  3 |   1953 |        9 |
|  4 |   1954 |        9 |
|  5 |   1955 |        7 |
|  6 |   1956 |        8 |
|  7 |   1957 |        8 |
|  8 |   1958 |       11 |
|  9 |   1959 |        9 |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique el campo seleccionado con su consulta y asegúrese de indicarlo en minúscula. Este debe ser:
  * `year`: año
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* La función `COUNT()` para este caso debe ser usada con el campo `raceId` como *parámetro*. Recuerde que esta función puede ser usada como `COUNT(*)` que es su caso genérico, y para casos más específicos como `COUNT(parámetro)` donde el parámetro puede variar dependiendo de la especificidad o necesidad de cada consulta.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* El ejercicio 3 y 4 usan la función `COUNT()` también, compare los ejercicios y encontrará similitudes y diferencias que le ayudarán a resolver este ejercicio.
</details>

## **6. Total de Puntos por Nacionalidad**

En este caso tomaremos en cuenta los datos de tres tablas: (`races`), (`drivers`) y (`results`) con la finalidad de obtener la información necesaria para resolver el ejercicio. La tabla (`drivers`) contiene la nacionalidad de los corredores, las cuales deben ser agrupadas para poder realizar el cálculo de la suma total de puntos obtenidos en las carreras.

> La cláusula `INNER JOIN` busca coincidencias entre 2 tablas en función a una columna que tengan en común. La sentencia `GROUP BY` se utiliza para organizar datos idénticos en grupos con la ayuda de algunas funciones auxiliares o complementarias.


En este ejercicio usted debe completar la función `points_year_nationality`, la cual deberá retornar una cadena de texto con una **consulta SQL válida** que contenga el total de puntos acumulados en la carreras por **nacionalidad**.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA points_year_country:

def points_year_nationality():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query

In [ ]:
from IPython.display import display
try:
    display(pd.read_sql(points_year_nationality(), conn).head(10))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>nationality</th>
      <th>points</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>American</td>
      <td>998.00</td>
    </tr>
    <tr>
      <th>1</th>
      <td>American-Italian</td>
      <td>0.00</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Argentine</td>
      <td>698.92</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Argentine-Italian</td>
      <td>0.00</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Australian</td>
      <td>2337.50</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Austrian</td>
      <td>990.50</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Belgian</td>
      <td>376.00</td>
    </tr>
    <tr>
      <th>7</th>
      <td>Brazilian</td>
      <td>3423.00</td>
    </tr>
    <tr>
      <th>8</th>
      <td>British</td>
      <td>8077.14</td>
    </tr>
    <tr>
      <th>9</th>
      <td>Canadian</td>
      <td>382.00</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Las consultas deben indicarse en minúsculas. Estas deben seguir el siguiente orden:
  * `nationality`: nacionalidad.
  * `points`: puntos totales por nacionalidad.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Por lo general, la función `GROUP BY` es utilizada con las funciones `COUNT()`, `MAX()`, `MIN()`, `SUM()` y/o `AVG()`. La que se debe utilizar en la resolución del ejercicio es `SUM()`.
</details>

## **7. Total de Puntos por Año y Conductor**
---
El modelo mostrado en la sección *Bases de datos* relaciona información de la *Formula 1* entre los años $1950$ y $2017$, cada tabla guarda información sobre determinada entidad, las entidades del modelo son conductores, resultados, carreras y circuitos, que al final se relacionan entre ellas mismas mediante `FOREIGN_KEYS`. Esta herramienta de `SQL` permite que la información guardada sea completa y debidamente distribuida.

Para este ejercicio es necesario usar determinados campos, no todos, de las tablas conductores (`drivers`), resultados (`results`) y carreras (`races`).

> Existe una sentencia de `SQL` que permite relacionar dos tablas mediante `FOREIGN_KEYS`, esta sentencia es `JOIN`. Esta, a su vez tiene algunas otras variaciones como `RIGHT JOIN`, `LEFT JOIN`, `CROSS JOIN`, `INNER JOIN` y `FULL OUTER JOIN` que también relacionan las tablas pero cada una lo hace de manera diferente, su trabajo es encontrar la sentencia **JOIN** adecuada que le permita retornar la busqueda solicitada exacta.


En este ejercicio usted deberá completar la función `points_year_driver()`, que retorne una cadena de texto con una **consulta SQL válida** que retorne los **PUNTOS** obtenidos por cada **AÑO** en **TODAS** las carreras ocurridas en cada año correspondiente por cada **CONDUCTOR**.

> Para este ejercicio usted debe hacer varias renombraciones con la sentencia `AS`, la renombración debe ser en minúscula y exactamente al nombre específicado:
* Concatene los campos `forename` y `surname` y luego esta concatenación la renombra como `full_name`.
* El campo año de la tabla carreras (`races.year`) como `year`.
* Los puntos obtenidos como `points`.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA points_year_driver:

def points_year_driver():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query

Use la siguiente celda para probar su código:

In [ ]:
from IPython.display import display
try:
    display(pd.read_sql(points_year_driver(), conn).head(10))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**

|    | full_name      |   year |   points |
|---:|:---------------|-------:|---------:|
|  0 | Adolf Brudes   |   1952 |        0 |
|  1 | Adolfo Cruz    |   1953 |        0 |
|  2 | Adrian Sutil   |   2007 |        1 |
|  3 | Adrian Sutil   |   2008 |        0 |
|  4 | Adrian Sutil   |   2009 |        5 |
|  5 | Adrian Sutil   |   2010 |       47 |
|  6 | Adrian Sutil   |   2011 |       42 |
|  7 | Adrian Sutil   |   2013 |       29 |
|  8 | Adrian Sutil   |   2014 |        0 |
|  9 | Adri��n Campos |   1987 |        0 |

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Verifique los campos seleccionados con su consulta y asegúrese de indicarlos en minúscula y renombrarlos correctamente. Estos deben ser y estar ordenados de la siguiente manera:
  1. **Concatenación** de `drivers.forename` junto `drivers.surname` -> **Renombramiento:** `full_name`.
  2. `races.year`: año carrera -> **Renombramiento:** `year`.
  3. **Total de puntos** de `results.points` -> **Renombramiento:** `points`.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* Analice las `FOREIGN_KEY` del modelo de la sección *Bases de datos*, esto le ayudará a saber entre qué tablas usar las sentencias `JOIN` correctamente.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 3</b></font>
</summary>

* Recuerde que para sumar todos los valores númericos de un campo se usa la función de agregación `SUM()` de `SQL` y dentro del argumento de la función se pasa el nombre de los campos. La petición de hallar el *Total de puntos* está relacionada con esta función.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 4</b></font>
</summary>

* La sentencia `GROUP BY` es necesaria en esta consulta, asegúrese de usarla en los **CAMPOS** correspondientes.
</details>

## **8. Piloto con más Puntos por Carrera**

Al igual que el ejercicio 6, se consideran los datos de las tablas (`races`), (`drivers`) y (`results`). La tabla (`drivers`) incluye toda la información necesaria de todos los corredores y, en conjunto con las tablas (`drivers`) y (`results`), se obtiene la información de la participación de cada uno en cada carrera.

En este ejercicio usted debe completar la función `higher_driver_race`, la cual deberá retornar una cadena de texto con una **consulta SQL válida** que contenga el **puntaje máximo** obtenido por un **píloto** por **carrera**.

**Parámetros de entrada:**

* No hay parámetros de entrada para esta función.

**Salida:**

* `query`: cadena de texto con una consulta válida de _SQLite_.

In [ ]:
# FUNCIÓN CALIFICADA higher_driver_race:

def higher_driver_race():
    """
    Retorna:
        query: cadena de texto con una consulta válida de SQLite.
    """
    ### ESCRIBA SU CÓDIGO AQUÍ ### (~1-9 líneas de código (cadena de texto multilínea))
    # Modifique la siguiente cadena de texto con la sintaxis
    # necesaria para realizar la consulta indicada.
    query = """
    SELECT
        ...
    FROM
      ...
    """
    return query

In [ ]:
from IPython.display import display
try:
    display(pd.read_sql(higher_driver_race(), conn).head(10))
except Exception as e:
    print("La consulta retornada no es válida. Revise su solución y guíese del siguiente error:")
    print(e)

**Salida esperada:**
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>full_name</th>
      <th>race_name</th>
      <th>race_year</th>
      <th>points</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Jenson Button</td>
      <td>Australian Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Jenson Button</td>
      <td>Malaysian Grand Prix</td>
      <td>2009</td>
      <td>5.0</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Sebastian Vettel</td>
      <td>Chinese Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Jenson Button</td>
      <td>Bahrain Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Jenson Button</td>
      <td>Spanish Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Jenson Button</td>
      <td>Monaco Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Jenson Button</td>
      <td>Turkish Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>7</th>
      <td>Sebastian Vettel</td>
      <td>British Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Mark Webber</td>
      <td>German Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
    <tr>
      <th>9</th>
      <td>Lewis Hamilton</td>
      <td>Hungarian Grand Prix</td>
      <td>2009</td>
      <td>10.0</td>
    </tr>
  </tbody>
</table>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 1</b></font>
</summary>

* Las consultas deben indicarse en minúsculas. Estas deben seguir el siguiente orden:
  * `full_name`: nombre **completo** del corredor.
  * `race_name`: nombre de la carrera.
  * `race_year`: año de la carrera.
  * `points`: puntos obtenidos por el corredor.
</details>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Pista 2</b></font>
</summary>

* SQL provee la función `CONCAT()` para unir cadenas dentro de una base de datos en una sola columna. Sin embargo, SQLite no tiene está función. En su lugar, utiliza el operador de concatenación `||` para unir dos cadenas en una.

    > <font size="2" color="darkblue"><i><b>Por ejemplo: </b></i></font>
    `'columna_1' || ' ' || 'columna_2';`
    
</details>

**¡Felicitaciones!** Ha terminado la tarea de conceptos de SQL de la Unidad 1. ¡Buen trabajo!

## **Entrega**

Para entregar el notebook por favor haga lo siguiente:
1. Descargue el notebook (`Archivo` -> `Descargar .ipynb`).
2. Ingrese a Edunext.
3. Realice el envío del *notebook* que descargó en la tarea (o quiz) correspondiente.
4. Recuerde que si tiene algún error, puede hacer múltiples intentos de envío en UNCode.

## **Créditos**
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Diseño, desarrollo del notebook y material audiovisual**

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*